In [1]:
import argparse, random
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader,Dataset
from torchvision import datasets, transforms, utils, models
from skimage import io, transform, img_as_float
from torch.autograd import Variable
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
import math
import random
from imblearn.over_sampling import RandomOverSampler
import pandas as pd
from lifelines.statistics import logrank_test
from lifelines.utils import concordance_index
import tables
import csv
import numpy as np
import json
from tqdm import tqdm
import gc
import copy
import os
from PIL import *
from PIL import ImageFile, Image
ImageFile.LOAD_TRUNCATED_IMAGES = False

In [2]:
csv_file='/deepdata/adib/prognostic_study/pytorch_code/Patch_clinical_new.csv'
files_list = pd.read_csv(csv_file)

/home/yuming/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
len(files_list)

975143

In [8]:
from PIL import Image
count = 0
for i in range(len(files_list)):
    if i%10000 == 0:
        print(str(i/len(files_list)*100)+"%")
    he_name = os.path.join(files_list.iloc[i, 0])
    he_name = he_name.replace('Teresa_TMA_patch', 'Patches')
    index_sub = he_name.find('Patches')
    he_name = os.path.join('/deepdata/adib/prognostic_study', he_name[index_sub:])
    he_name = he_name.replace('\\', '/')  
    try:
        img = Image.open(he_name) # open the image file
        img.verify() # verify that it is, in fact an image
    except (IOError, SyntaxError) as e:
        count = count + 1
        if os.path.isfile(he_name):
            os.remove(he_name)
        files_list.drop(files_list.index[[i]], inplace=True)           
        print('Bad file:', files_list.iloc[i, 0]) # print out the names of corrupt files
print(count)

0.0%
1.0254906203500411%
Bad file: F:\Prognostic study\Patches\deceased\All_A5_21600_rot180_res1.jpeg
Bad file: F:\Prognostic study\Patches\deceased\All_A5_31500_rot180_res1.jpeg
2.050985447232759%
Bad file: F:\Prognostic study\Patches\deceased\All_A9_215628_rot180_res1.jpeg
Bad file: F:\Prognostic study\Patches\deceased\All_A9_217440_rot180_res1.jpeg
Bad file: F:\Prognostic study\Patches\deceased\All_A9_218950_rot180_res1.jpeg
Bad file: F:\Prognostic study\Patches\deceased\All_A9_219856_rot180_res1.jpeg
Bad file: F:\Prognostic study\Patches\deceased\All_A9_221970_rot180_res1.jpeg
Bad file: F:\Prognostic study\Patches\deceased\All_A9_226500_rot180_res1.jpeg
3.076497100401483%
4.101996133868644%
5.127495167335805%
Bad file: F:\Prognostic study\Patches\deceased\All_C17_670680_rot180_res1.jpeg
Bad file: F:\Prognostic study\Patches\deceased\All_C17_860220_rot180_res1.jpeg
Bad file: F:\Prognostic study\Patches\deceased\All_C17_864432_rot180_res1.jpeg
6.15301313053002%
Bad file: F:\Prognosti

IndexError: single positional indexer is out-of-bounds

In [9]:
len(files_list)

975102

In [10]:
files_list.to_csv(r'/deepdata/adib/prognostic_study/pytorch_code/Patch_clinical_clean_v2.csv', encoding='utf-8', index=False)

In [2]:
cwd = os.getcwd()
surPath = os.path.join(cwd, '../Patches', 'survival')
decPath = os.path.join(cwd, '../Patches', 'deceased')
fileListSur = [name for name in os.listdir(surPath) if 
                    os.path.isfile(os.path.join(surPath, name))]
fileListDec = [name for name in os.listdir(decPath) if 
                    os.path.isfile(os.path.join(decPath, name))]  

In [11]:
len(fileListSur)
len(fileListDec)

1186514

In [9]:
count_1 = 0
for index, item in enumerate(fileListSur):
    if index%10000 == 0:
        print(str(index/len(fileListSur)*100)+"%")
    try:
        img = io.imread(os.path.join(surPath, item)) # open the image file
#         img.verify() # verify that it is, in fact an image
    except (IOError, SyntaxError, ValueError) as e:
        fileListSur.pop(index)
        count_1 = count_1 + 1
        print('Bad file:', item) # print out the names of corrupt files

count_2 = 0
for index, item in enumerate(fileListDec):
    if index%10000 == 0:
        print(str(index/len(fileListDec)*100)+"%")
    try:
        img = io.imread(os.path.join(decPath, item)) # open the image file
#         img.verify() # verify that it is, in fact an image
    except (IOError, SyntaxError, ValueError) as e:
        fileListDec.pop(index)
        count_2 = count_2 + 1
        print('Bad file:', item) # print out the names of corrupt files


0.0%
3.056524303953003%
Bad file: All_D13_248912_rot90_res1.jpeg
6.113067292644757%
Bad file: All_D13_63552_rot180_res1.jpeg
9.169628966246595%
12.226171954995461%
15.282714943744327%
18.33925793249319%
21.395800921242056%
24.452343909990923%
27.50888689873979%
30.565429887488655%
33.621972876237514%
36.67851586498638%
39.735058853735254%
42.79160184248411%
45.84814483123298%
48.904687819981845%
51.961230808730704%
55.01777379747958%
58.074316786228444%
61.13085977497731%
64.18740276372617%
67.24394575247503%
70.3004887412239%
73.35703172997276%
76.41357471872163%
79.47011770747051%
Bad file: HPan-Ade170Sur-01_115_I3_12375_rot90_res0.5.jpeg
Bad file: HPan-Ade170Sur-01_115_I3_141900_rot45_res1.jpeg
Bad file: HPan-Ade170Sur-01_115_I3_18810_rot90_res1.jpeg
Bad file: HPan-Ade170Sur-01_115_I3_190080_rot180_res1.jpeg
Bad file: HPan-Ade170Sur-01_115_I3_190575_rot90_res1.jpeg
Bad file: HPan-Ade170Sur-01_115_I3_217140_rot180_res1.jpeg
Bad file: HPan-Ade170Sur-01_115_I3_21780_rot180_res0.5.jpeg


In [10]:
print(count_1+count_2)

63


In [12]:
from itertools import cycle
csvFilePath = os.path.join(cwd, 'image_files.csv')

fileListSur.sort()
fileListDec.sort()

numOfSur = len(fileListSur)
numOfDec = len(fileListDec)
numOfPatch = max(numOfSur, numOfDec)
zipList = zip(cycle(fileListSur), fileListDec)
zipList = list(zipList)
print(numOfPatch)
print(zipList[0])

with open(csvFilePath, 'w', newline='') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',', quotechar='|', 
                            quoting=csv.QUOTE_MINIMAL)
    for i in range(numOfPatch):

        sur = os.path.join(surPath, str(zipList[i][0]))
        dec = os.path.join(decPath, str(zipList[i][1]))
        filewriter.writerow([sur, dec])

1186514
('All_A11_1001112_rot45_res1.jpeg', 'All_A13_1000160_rot45_res1.jpeg')


In [35]:
class LambdaLR:
    def __init__(self, n_epochs, offset, decay_start_epoch):
        assert (n_epochs - decay_start_epoch) > 0, "Decay must start before the training session ends!"
        self.n_epochs = n_epochs
        self.offset = offset
        self.decay_start_epoch = decay_start_epoch

    def step(self, epoch):
        print(epoch)
        print("test")
        return 1.0 - max(0, epoch + self.offset - self.decay_start_epoch) / (self.n_epochs - self.decay_start_epoch)

In [37]:
import torch
from torchvision import datasets, transforms, utils, models
model_ft = models.resnet101()  
optimizer = torch.optim.Adam(model_ft.parameters(), lr=0.002)
lr = torch.optim.lr_scheduler.LambdaLR(
    optimizer, lr_lambda=LambdaLR(200, 0, 5).step
)

0
test
